In [1]:
import pandas as pd
import numpy as np

from numba import njit

In [2]:
target_url = 'https://coral.ise.lehigh.edu/wp-content/uploads/2014/07/data.d/tai12a.dat'
    
dd = pd.read_table(target_url,sep='\s+').reset_index()

n = int(dd.shape[0]/2)

flow = dd.iloc[:n].values

dist = dd.iloc[n:].values

In [3]:
opt_sol = np.array([8,1,6,2,11,10,3,5,9,7,12,4])

opt_val = 224416

In [4]:
def cost(flow, dist, sol):
    obj = 0
    for i in range(n):
        for j in range(n):
            obj = obj + dist[sol[i]-1][sol[j]-1]*flow[i][j]

    return obj

In [5]:
jitted_cost = njit()(cost)

In [6]:
%%time

cost(flow, dist, opt_sol)

CPU times: user 319 µs, sys: 1e+03 ns, total: 320 µs
Wall time: 323 µs


224416

In [7]:
%%time

jitted_cost(flow, dist, opt_sol)

CPU times: user 529 ms, sys: 75.9 ms, total: 605 ms
Wall time: 325 ms


224416

In [61]:
def rndsol(n):
    random_sol = np.arange(1,n+1)
    np.random.shuffle(random_sol)

    return random_sol

In [62]:
%%time

rndsol(n)

CPU times: user 35 µs, sys: 5 µs, total: 40 µs
Wall time: 44.1 µs


array([ 5,  6, 10, 11,  3, 12,  2,  7,  4,  9,  8,  1])

In [72]:
jitted_rndsol = njit()(rndsol)

In [74]:
%%time

jitted_rndsol(n)

CPU times: user 13 µs, sys: 0 ns, total: 13 µs
Wall time: 16 µs


array([ 3,  4,  1,  2,  6,  8, 12,  7, 11, 10,  5,  9])

In [52]:
def swap(i,j,sol):
    new_sol = sol.copy()
    a = sol[i]
    b = sol[j]
    new_sol[i] = b
    new_sol[j] = a
    
    return new_sol

In [53]:
init_sol = np.arange(1,n+1)

In [54]:
%%time
swap(0,9,init_sol)

CPU times: user 16 µs, sys: 4 µs, total: 20 µs
Wall time: 23.8 µs


array([10,  2,  3,  4,  5,  6,  7,  8,  9,  1, 11, 12])

In [58]:
jitted_swap = njit()(swap)

In [60]:
%%time
jitted_swap(0,9,init_sol)

CPU times: user 13 µs, sys: 0 ns, total: 13 µs
Wall time: 16.9 µs


array([10,  2,  3,  4,  5,  6,  7,  8,  9,  1, 11, 12])

In [35]:
def local_search(sol):
    current_sol = sol.copy()
    current_cost = cost(flow, dist, current_sol)

    current_best = current_cost
    best_sol = current_sol.copy()

    for i in range(n):
        for j in range(i+1,n):

            new_sol = swap(i,j, current_sol)
            new_cost = cost(flow, dist, new_sol)

            if new_cost<current_best:
                current_best = new_cost
                best_sol = new_sol.copy()
                best_pair = i,j

    return best_sol

In [36]:
def jitted_local_search(sol):
    current_sol = sol.copy()
    current_cost = jitted_cost(flow, dist, current_sol)

    current_best = current_cost
    best_sol = current_sol.copy()

    for i in range(n):
        for j in range(i+1,n):

            new_sol = jitted_swap(i,j, current_sol)
            new_cost = jitted_cost(flow, dist, new_sol)

            if new_cost<current_best:
                current_best = new_cost
                best_sol = new_sol.copy()
                best_pair = i,j

    return best_sol

In [37]:
init_sol = np.arange(1,n+1)

In [38]:
%%time
new_sol = local_search(init_sol)

CPU times: user 22.2 ms, sys: 690 µs, total: 22.8 ms
Wall time: 22.1 ms


In [40]:
%%time
new_sol = jitted_local_search(init_sol)

CPU times: user 207 µs, sys: 23 µs, total: 230 µs
Wall time: 211 µs


In [86]:
def calculate_delta(flow, dist, sol, i, j):
    
    d = (flow[i][i]-flow[j][j])*(dist[sol[j]-1][sol[j]-1]-dist[sol[i]-1][sol[i]-1]) + \
        (flow[i][j]-flow[j][i])*(dist[sol[j]-1][sol[i]-1]-dist[sol[i]-1][sol[j]-1])
        
    for k in range(n):
        if k!=i and k!=j:
            d = d + (flow[k][i]-flow[k][j])*(dist[sol[k]-1][sol[j]-1]-dist[sol[k]-1][sol[i]-1]) + \
                    (flow[i][k]-flow[j][k])*(dist[sol[j]-1][sol[k]-1]-dist[sol[i]-1][sol[k]-1])
            
    return d

In [89]:
sol = np.arange(1,n+1)

delta = np.zeros(n*n).reshape(n,n)

for i in range(n):
    for j in range(n):
        delta[i][j] = calculate_delta(flow, dist, sol, i, j)
        


np.argwhere(delta == np.min(delta))[0]

array([0, 9])

In [43]:
def Grasp(it=100):
    init_sol = rndsol(n)
    
    current_sol = init_sol.copy()
    current_cost = cost(flow, dist, init_sol)
    
    best_sol = init_sol.copy()
    best_cost = current_cost
    
    for i in range(it):
        current_sol = local_search(current_sol)
        current_cost = cost(flow, dist, current_sol)
        
        if current_cost < best_cost:
            best_sol = current_sol.copy()
            best_cost = current_cost
            
    return best_sol

In [49]:
%%time
sol = Grasp(1000)

CPU times: user 16.7 s, sys: 230 ms, total: 16.9 s
Wall time: 16.9 s


In [48]:
def jitted_Grasp(it=100):
    init_sol = jitted_rndsol(n)
    
    current_sol = init_sol.copy()
    current_cost = jitted_cost(flow, dist, init_sol)
    
    best_sol = init_sol.copy()
    best_cost = current_cost
    
    for i in range(it):
        current_sol = jitted_local_search(current_sol)
        current_cost = jitted_cost(flow, dist, current_sol)
        
        if current_cost < best_cost:
            best_sol = current_sol.copy()
            best_cost = current_cost
            
    return best_sol

In [78]:
%%time
sol = jitted_Grasp(100000)

CPU times: user 11.9 s, sys: 196 ms, total: 12.1 s
Wall time: 12 s


In [79]:
sol

array([ 5,  9,  7,  6,  4, 11,  1, 10,  8, 12,  2,  3])

In [80]:
jitted_cost(flow, dist, sol)

252448